In [7]:
from __future__ import print_function
import keras
import numpy as np
from keras.utils import Sequence
from sklearn.utils import shuffle
import re

In [8]:
class TweetSeqGenerator(Sequence):
  
    def __init__(self, data, labels, batch_size, normalize=False):
      self.data = data
      self.labels = labels
      if len(self.data) != len(self.labels):
        raise ValueError("Data and Labels have to be same size")
      self.batch_size = batch_size
      self.normalize = normalize
      
      # If normalize, then normalize each timeseries to sum to 1 on each keyword
      if self.normalize:
        for i in range(len(self.data)):
            # Normalizes the rows of the matrix to 1
            # Axis=0 should normalize the columns to 1
            self.data[i] = self.data[i]/np.max(self.data[i], axis=1)


    def __getitem__(self, index):
        """
        Go through each set of files and augment them as needed
        :param index:
        :return:
        """
        timeseries = self.data[index * self.batch_size:(index + 1) * self.batch_size]
        labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        return timeseries, labels

    def __len__(self):
        """
        Returns the length of the list of paths, as the number of events is not known
        :return:
        """
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def on_epoch_end(self):
            self.data = shuffle(self.data, self.labels)


In [9]:
def generate_timeseries(category, length, limit=None):
  """
  Generates timeseries from the tweets, and gets the next event to occur in the future
  """
  return NotImplementedError

In [10]:
def clean_tweet(tweet):
  """
  Cleans a tweet to just English characters, removing URLs, etc.
  What is given is just the tweet itself, not the metadata
  """
  tweet = re.sub(r"http\S+", "", tweet) # removes URLs
  tweet = re.sub(r"[^a-zA-Z0-9]+", ' ', tweet) # Removes non-alphanumeric chars
  tweet = tweet.lower() # Lowercases it
  
  return tweet
  
  

In [14]:
import keras
from keras.utils import Sequence
from sklearn.utils import shuffle
import re

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import pandas as pd
import google.datalab.bigquery as bq
    
def getDATA():
    query_K2019tweets = """

    SELECT created_at,keywords FROM `hdc-politie-team-3.politie.tweets` WHERE iso_language_code='nl' and created_at BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY created_at ASC

    """
    print(query_K2019tweets)
    # Transform your query to BigQuery object
    bq_object = bq.Query(query_K2019tweets)

    # Execute your query
    result = bq_object.execute().result()

    # Transform your output to a Pandas dataframe
    kings2019tweets_df = result.to_dataframe()
    
    return kings2019tweets_df
  

def getLabels():
  query_K2019tweets = """

  SELECT * FROM `hdc-politie-team-3.politie.meldkamer` WHERE start_incident BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY start_incident ASC

  """
  print(query_K2019tweets)
  # Transform your query to BigQuery object
  bq_object = bq.Query(query_K2019tweets)

  # Execute your query
  result = bq_object.execute().result()

  # Transform your output to a Pandas dataframe
  kings2019mcl_df = result.to_dataframe()

  return kings2019mcl_df


def getkeywords():
    query_K2019keys = """

   SELECT distinct key FROM `hdc-politie-team-3.politie.tweets`,unnest(keywords) as key

    """
    # Transform your query to BigQuery object
    bq_object = bq.Query(query_K2019keys)
    
    # Execute your query
    result = bq_object.execute().result()

    # Transform your output to a Pandas dataframe
    kings2019keys_df = result.to_dataframe()
    aj=kings2019keys_df.values.tolist()
    newa=[]
    for i in range(len(aj)):
        newa.append(aj[i][0])
    
    nexta=sorted(newa)
    
    return nexta
  
def getvectorfortime(b,a):
    newb=b.values.tolist()
    c=[0]*len(a)
    for i in range (len(newb)):
        am= newb[i][1]
        
        for j in range (len(am)):
            c[a.index(am[j])]+=1
    return c
        

keywords=getkeywords()
data=getDATA()
labels=getLabels()

def create_timestep(start_time, step_size, length, incident_type='Brand'):
  """
  
  step_size has to be a pd.timeDelta to work, otherwise it will fail
  
  Returns the timeseries and the labels for the timeseries
  
  """
  timeseries = []
  for i in range(length):
    mask = (data['created_at'] >= start_time+i*step_size) & (data['created_at'] <= start_time+(i+1)*step_size)
    sub_data = data[mask]
    timeseries.append(getvectorfortime(sub_data,keywords))
  
  timeseries = np.asarray(timeseries)
  end_time = start_time+length*step_size
  # 1min, 5min, 10min, 15min, 30min
  time_periods = [pd.Timedelta("1 minute"), pd.Timedelta("5 minute"), pd.Timedelta("10 minute"), pd.Timedelta("15 minute"), pd.Timedelta("30 minute")]
  time_label = []
  for period in time_periods:
    label_mask = (labels['start_incident'] >= (end_time)) & (labels['start_incident'] <= (end_time+period)) & (labels['mcl'] == incident_type)
    if len(labels[label_mask]) > 0:
      time_label.append(1)
    else:
      time_label.append(0)
    
  return timeseries, time_label
  


class TweetSeqGenerator(Sequence):

    def __init__(self, data, labels, batch_size, normalize=False):
        self.data = data
        self.labels = labels
        if len(self.data) != len(self.labels):
            raise ValueError("Data and Labels have to be same size")
        self.batch_size = batch_size
        self.normalize = normalize

        # If normalize, then normalize each timeseries to sum to 1 on each keyword
        if self.normalize:
            for i in range(len(self.data)):
                # Normalizes the rows of the matrix to 1
                # Axis=0 should normalize the columns to 1
                self.data[i] = self.data[i]/np.max(self.data[i], axis=1)

    def __getitem__(self, index):
        """
        Go through each set of files and augment them as needed
        :param index:
        :return:
        """
        timeseries = self.data[index * self.batch_size:(index + 1) * self.batch_size]
        labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        return timeseries, labels

    def __len__(self):
        """
        Returns the length of the list of paths, as the number of events is not known
        :return:
        """
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def on_epoch_end(self):
        self.data, self.labels = shuffle(self.data, self.labels)


def generate_timeseries(category, length, limit=None):
    """
    Generates timeseries from the tweets, and gets the next event to occur in the future
    """
    return NotImplementedError


def clean_tweet(tweet):
    """
    Cleans a tweet to just English characters, removing URLs, etc.
    What is given is just the tweet itself, not the metadata
    """
    tweet = re.sub(r"http\S+", "", tweet) # removes URLs
    tweet = re.sub(r"[^a-zA-Z0-9]+", ' ', tweet) # Removes non-alphanumeric chars
    tweet = tweet.lower() # Lowercases it

    return tweet


step_size = pd.Timedelta("1 minute")
start_time = pd.to_datetime("2019-04-26 00:00:00")
length = 30
tweet_keywords = []
melding = []
for i in range(1000):
  timeseries, label = create_timestep(start_time, step_size, length=length)
  tweet_keywords.append(timeseries)
  melding.append(label)
  start_time += step_size * (length/15)




    SELECT created_at,keywords FROM `hdc-politie-team-3.politie.tweets` WHERE iso_language_code='nl' and created_at BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY created_at ASC

    


  SELECT * FROM `hdc-politie-team-3.politie.meldkamer` WHERE start_incident BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY start_incident ASC

  


In [15]:
maxlen = length
sequences = tweet_keywords
next_melding = melding

x = np.asarray(sequences)
y = np.asarray(next_melding)

x, y = shuffle(x,y)

validation_cut = int(0.25*len(x))
x1 = x[:validation_cut]
y1 = y[:validation_cut]

x = x[validation_cut:]
y = y[validation_cut:]

print("X Hsape: {}".format(x.shape))
print("Y Shape: {}".format(y.shape))

X Hsape: (750, 30, 1100)
Y Shape: (750, 5)


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(tweet_keywords[0][0]))))
model.add(LSTM(128))
model.add(Dense(len(melding[0])))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

train_gen = TweetSeqGenerator(data=x, labels=y, batch_size=32)
val_gen = TweetSeqGenerator(data=x1, labels=y1, batch_size=32)

model.fit_generator(generator=train_gen, validation_data=val_gen, use_multiprocessing=True, workers=4, epochs=1000)


def predict_melding():
    for i in range(0,len(sequences)-maxlen):
        x_pred = np.array(sequences[i:i+maxlen])
        pred = model.predict(x_pred)
        print()
        for j in range(len(pred)):
            print("time = " + str(i) + " predictions for time = "+str(i+j+1)+ ":")
            for k in range(len(pred[0])):
                print("indicent " + str(k) + ": " + str(round(pred[0][k]* 100,2)) + "%")

predict_melding()